# Calculate post-TT dM/M0 and CBF time series for every ROI

Calculate the perfusion (dM/M0) time series  (i.e. vs PLD) for each ROI and filter only PLDs that are longer than the arterial transit time. Then calculate CBF using the white paper model, substituting assumed vs calculated T1 of arterial blood (aka T1a) parameter values. The best T1blood value should provide a flat response for PLD>ATT. That is because once the label reaches the voxel (PLD>TT) it just sits there and relaxes with T1b, so adjusting the CBF calculation for PLD with the correct T1a (arterial blood T1) should result in the same CBF regardless of PLD.


In [1]:
import os
import sys
import pandas as pd
import numpy as np
import subprocess
import glob
import flywheel

local_dir = '../data/subjects'

In [2]:
def refine_list(lst):
    '''
    creates a usable list of floats from the fsl output string

    :param lst: fsl output string
    :return: list of floats
    
    '''
    result_list = str(lst).split(' ')
    result_list[0] = result_list[0].replace("b'", "")
    result_list = result_list[: len(result_list) - 1] # remove extra text
    result_list = [float(e) for e in result_list]
    
    return(result_list)
    

In [3]:
cols = ['session_id', 'gray_matter','acc', 'insula', 'caudate', 'putamen']
df = pd.DataFrame(columns=cols)
for root, dirs, files in os.walk(local_dir, topdown=False):
    pw_maps = [f for f in files if 'dMM0' in f]
    if len(pw_maps) > 0:
        for pw_map in pw_maps:
            ses_name = pw_map.split('dMM0')[0][:-1] # drop the last underscore
            pw_map_path = os.path.join(root, pw_map)
            roi_dir = os.path.join(root, ses_name + "_rois")
            
            for mask in glob.glob(roi_dir+"/*roimask.nii.gz"):
                cmd1="fslmeants --transpose -i '{}' -m '{}'".format(pw_map_path, mask)
                result = refine_list(subprocess.check_output(cmd1, shell=True))
                if 'acc' in mask:
                    acc = result
                elif 'insula' in mask:
                     insula = result
                elif 'caudate' in mask:
                    caudate = result
                elif 'putamen' in mask:
                    putamen = result
                elif 'GM' in mask:
                    gray_matter = result

            
            
            df.loc[len(df.index)] = [ses_name, gray_matter, acc, insula, caudate, putamen]

df.head()

,session_id,gray_matter,acc,insula,caudate,putamen
0,sub-HC015_ses-MR_FOLLOWUP,"[0.03326, 0.036367, 0.030014, 0.024612, 0.0183...","[0.033975, 0.031439, 0.024474, 0.020017, 0.009...","[0.035437, 0.029707, 0.023442, 0.013236, 0.008...","[0.020504, 0.017798, 0.013022, 0.009686, 0.006...","[0.026987, 0.023193, 0.016136, 0.01437, 0.0090..."
1,sub-HC025_ses-MR_BASELINE,"[0.034164, 0.040535, 0.029705, 0.030699, 0.026...","[0.032157, 0.028815, 0.021619, 0.019555, 0.018...","[0.03424, 0.024612, 0.017674, 0.018719, 0.0170...","[0.015555, 0.014892, 0.009294, 0.009135, 0.009...","[0.02363, 0.01853, 0.013335, 0.01409, 0.010391..."
2,sub-HC025_ses-MR_FOLLOWUP,"[0.026898, 0.028915, 0.027759, 0.018698, 0.014...","[0.026438, 0.02391, 0.022244, 0.012148, 0.0109...","[0.032457, 0.023377, 0.022453, 0.013957, 0.009...","[0.008436, 0.012984, 0.010903, 0.006265, 0.004...","[0.022181, 0.017441, 0.016612, 0.010571, 0.007..."
3,sub-HC005_ses-MR_BASELINE,"[0.025094, 0.036176, 0.030915, 0.028319, 0.021...","[0.023494, 0.036394, 0.025192, 0.023385, 0.013...","[0.034577, 0.028567, 0.021199, 0.02043, 0.0122...","[0.008906, 0.019715, 0.011383, 0.012684, 0.004...","[0.019445, 0.022601, 0.015416, 0.014497, 0.007..."
4,sub-HC005_ses-MR_FOLLOWUP,"[0.021077, 0.03066, 0.022244, 0.017955, 0.0194...","[0.026329, 0.030378, 0.021136, 0.013982, 0.017...","[0.032553, 0.028926, 0.018105, 0.013776, 0.016...","[0.016474, 0.018874, 0.012538, 0.007039, 0.010...","[0.01589, 0.018897, 0.011758, 0.009262, 0.0101..."


In [4]:
# example
list(df.loc[df['session_id']=='sub-HC001_ses-MR_BASELINE', 'acc'])[0]

[0.015712, 0.022783, 0.022847, 0.015866, 0.013916, 0.014482, 0.007363]

## Select time points based on transit time for that region
Take the time series list from the above calculation and exclude time points (i.e. PLDs) that happen before ATT (i.e. when all the blood has arrived). The PLDs in this experiment are [1, 1.22, 1.48, 1.78, 2.15,x, y]. We can determine which to keep based on how long the transit-time was using a decision tree. E.g. if the transit time was between the first (1s) and second (1.22s) delays, then we eliminate the first from the data. We do this by adding a NaN at the end, so that the resulting time series is PLDs after ATT (i.e. first PLD after, second PLD after) rather than the absolute (first (1s), second (1.22s), etc.)

In [5]:
# Check the maximum transit-time
tt_df = pd.read_csv("regionals_tts.csv")
np.max(tt_df)

session_id     sub-HC028_ses-MR_FOLLOWUP
gray_matter                  1793.280349
acc                          1885.564396
insula                       1533.890533
caudate                      1524.748148
putamen                      1624.468193
dtype: object

It looks like the maximum transit time in the dataset is not more than the fifth PLD (2.15s). So there should be at least 3 points in every time series.

In [6]:
refined_df = df.copy()

for row, row in df.iterrows(): # iterate over rows
    row_list = [row.session_id] 
    ses_id = row.session_id
    for c, value in row.items():
        if type(value) == list:
            tt = float(tt_df.loc[tt_df['session_id']==row.session_id, c])/1000 # originally in ms, but we want s
            
            # extract post-TT values
            if tt < 1:
                dms = value[0:7] # all 7
            elif 1 < tt < 1.22:
                dms = [np.nan] + value[1:7] 
            elif 1.22 < tt < 1.48:
                dms = [np.nan]*2 + value[2:7]  
            elif 1.48 < tt < 1.78:
                dms = [np.nan]*3 + value[3:7]  
            elif 1.78 < tt < 2.15:
                dms = [np.nan]*4 + value[4:7] # see above comment
            else:
                print("??")

            row_list.append(dms)

    refined_df.loc[len(refined_df.index)] = row_list
    
refined_df = refined_df.tail(50) # only get the newly created rows
refined_df = refined_df.reset_index(drop=True)
refined_df.head()

,session_id,gray_matter,acc,insula,caudate,putamen
0,sub-HC015_ses-MR_FOLLOWUP,"[nan, 0.036367, 0.030014, 0.024612, 0.018347, ...","[nan, 0.031439, 0.024474, 0.020017, 0.009921, ...","[nan, 0.029707, 0.023442, 0.013236, 0.008785, ...","[0.020504, 0.017798, 0.013022, 0.009686, 0.006...","[nan, 0.023193, 0.016136, 0.01437, 0.009061, 0..."
1,sub-HC025_ses-MR_BASELINE,"[nan, 0.040535, 0.029705, 0.030699, 0.026136, ...","[nan, 0.028815, 0.021619, 0.019555, 0.018986, ...","[nan, 0.024612, 0.017674, 0.018719, 0.017005, ...","[nan, 0.014892, 0.009294, 0.009135, 0.009696, ...","[nan, 0.01853, 0.013335, 0.01409, 0.010391, 0...."
2,sub-HC025_ses-MR_FOLLOWUP,"[nan, nan, 0.027759, 0.018698, 0.014911, 0.013...","[nan, 0.02391, 0.022244, 0.012148, 0.010913, 0...","[nan, 0.023377, 0.022453, 0.013957, 0.00939, 0...","[nan, 0.012984, 0.010903, 0.006265, 0.004185, ...","[nan, 0.017441, 0.016612, 0.010571, 0.007251, ..."
3,sub-HC005_ses-MR_BASELINE,"[nan, nan, 0.030915, 0.028319, 0.021014, 0.018...","[nan, 0.036394, 0.025192, 0.023385, 0.013777, ...","[nan, 0.028567, 0.021199, 0.02043, 0.012227, 0...","[nan, 0.019715, 0.011383, 0.012684, 0.004337, ...","[nan, 0.022601, 0.015416, 0.014497, 0.00756, 0..."
4,sub-HC005_ses-MR_FOLLOWUP,"[nan, nan, 0.022244, 0.017955, 0.0194, 0.01465...","[nan, 0.030378, 0.021136, 0.013982, 0.017686, ...","[nan, 0.028926, 0.018105, 0.013776, 0.016434, ...","[nan, 0.018874, 0.012538, 0.007039, 0.01058, 0...","[nan, nan, 0.011758, 0.009262, 0.010172, 0.006..."


## Pull Hematocrit data from Flywheel
The hematocrit data is essential for calculating T1a. It lives on Flywheel in each subject's custom info. 

In [7]:
fw = flywheel.Client()
project_id = '5e50277c6dea314fa72a7440'
project = fw.get(project_id)

# initialize target columns with NaNs
refined_df['Hb'] = np.nan
refined_df['T1a'] = np.nan

sessions = [fw.get_session(x.id) for x in fw.get_project_sessions(project_id)]

for ses in sessions:
    subject = fw.get(ses.parents['subject'])
    session_id = "sub-{}_ses-{}".format(subject.label, ses.label) # NOTE this is not the Flywheel definition of 'id'

    if subject.info:
        hb = subject.info['Hb']
        t1a = hb / 3 #??  conversion equation
        refined_df.loc[refined_df['session_id']==session_id, 'Hb'] = hb
        refined_df.loc[refined_df['session_id']==session_id, 'T1a'] = t1a 

refined_df.to_csv('refined_dM.csv', index=False)
refined_df.head()

,session_id,gray_matter,acc,insula,caudate,putamen,Hb,T1a
0,sub-HC015_ses-MR_FOLLOWUP,"[nan, 0.036367, 0.030014, 0.024612, 0.018347, ...","[nan, 0.031439, 0.024474, 0.020017, 0.009921, ...","[nan, 0.029707, 0.023442, 0.013236, 0.008785, ...","[0.020504, 0.017798, 0.013022, 0.009686, 0.006...","[nan, 0.023193, 0.016136, 0.01437, 0.009061, 0...",14.8,4.933333
1,sub-HC025_ses-MR_BASELINE,"[nan, 0.040535, 0.029705, 0.030699, 0.026136, ...","[nan, 0.028815, 0.021619, 0.019555, 0.018986, ...","[nan, 0.024612, 0.017674, 0.018719, 0.017005, ...","[nan, 0.014892, 0.009294, 0.009135, 0.009696, ...","[nan, 0.01853, 0.013335, 0.01409, 0.010391, 0....",15.9,5.300000
2,sub-HC025_ses-MR_FOLLOWUP,"[nan, nan, 0.027759, 0.018698, 0.014911, 0.013...","[nan, 0.02391, 0.022244, 0.012148, 0.010913, 0...","[nan, 0.023377, 0.022453, 0.013957, 0.00939, 0...","[nan, 0.012984, 0.010903, 0.006265, 0.004185, ...","[nan, 0.017441, 0.016612, 0.010571, 0.007251, ...",15.9,5.300000
3,sub-HC005_ses-MR_BASELINE,"[nan, nan, 0.030915, 0.028319, 0.021014, 0.018...","[nan, 0.036394, 0.025192, 0.023385, 0.013777, ...","[nan, 0.028567, 0.021199, 0.02043, 0.012227, 0...","[nan, 0.019715, 0.011383, 0.012684, 0.004337, ...","[nan, 0.022601, 0.015416, 0.014497, 0.00756, 0...",12.2,4.066667
4,sub-HC005_ses-MR_FOLLOWUP,"[nan, nan, 0.022244, 0.017955, 0.0194, 0.01465...","[nan, 0.030378, 0.021136, 0.013982, 0.017686, ...","[nan, 0.028926, 0.018105, 0.013776, 0.016434, ...","[nan, 0.018874, 0.012538, 0.007039, 0.01058, 0...","[nan, nan, 0.011758, 0.009262, 0.010172, 0.006...",12.2,4.066667


## Calculate CBF


In [8]:
def calc_cbf_ts(dMM0_list, T1blood):
    '''
    calculates CBF values from list of DM/M0 values

    :param dMM0_list:  the list of dM/M0 values
    :param T1blood: the assumed (1.65) or calculated T1 relaxation time of blood
    :return: list of CBF values
    
    '''
    
    plds = [1, 1.22, 1.48, 1.78, 2.15, 2.63, 3.32] # post-labeling durations
    cbf_ts = []
    
    # set the parameters
    lmbda = 0.9
    alpha = 0.85
    tau = 3.5 # labeling duration
    
    denominator = 2*alpha*T1blood*(1-np.exp(-tau/T1blood)) # only need to calculate once                         
                                   
    for i in range(len(dMM0_list)-1):
        # calculate CBF factor based on corresponding PLD value
        pld = plds[i]                                   
        numerator = 6000 * lmbda * np.exp(pld/T1blood)
        factor = numerator/denominator                           
        # multiply by deltaM over M0 to get the CBF value for this PLD                          
        cbf = dMM0_list[i] * factor
        cbf_ts.append(cbf) 
                                
    return cbf_ts
    

In [9]:
cbf_df = refined_df.copy()
for row, row in refined_df.iterrows(): #iterate over rows
    row_list = [row.session_id]
    for c, value in row.items():
        if type(value) == list:
            cbf_ts = calc_cbf_ts(value, 1.65) 
            row_list.append(cbf_ts)
    
    
    row_list.append(row.Hb)
    row_list.append(row.T1a)
#     print(row_list)
    cbf_df.loc[len(cbf_df.index)] = row_list

cbf_df = cbf_df.tail(50) # only get the newly created rows
cbf_df = cbf_df.reset_index(drop=True)
# cbf_df.to_csv('cbf_ts.csv', index=False)
cbf_df.head()

,session_id,gray_matter,acc,insula,caudate,putamen,Hb,T1a
0,sub-HC015_ses-MR_FOLLOWUP,"[nan, 166.62657198424628, 160.98853314340246, ...","[nan, 144.0474275198042, 131.27318451894556, 1...","[nan, 136.11173794747998, 125.73776217590593, ...","[82.21848033729223, 81.5469994273824, 69.84716...","[nan, 106.26584771992806, 86.54997570473586, 9...",14.8,4.933333
1,sub-HC025_ses-MR_BASELINE,"[nan, 185.7235431952436, 159.33112470929464, 1...","[nan, 132.02476618159477, 115.95958879280394, ...","[nan, 112.7674317286625, 94.79947140589374, 12...","[nan, 68.23226853986844, 49.850983775397566, 5...","[nan, 84.90088208727923, 71.52602417096261, 90...",15.9,5.300000
2,sub-HC025_ses-MR_FOLLOWUP,"[nan, nan, 148.89320622135367, 120.28981407315...","[nan, 109.5510032761385, 119.31195213040063, 7...","[nan, 107.1089001918147, 120.43298243049296, 8...","[nan, 59.49018095095702, 58.481307951706434, 4...","[nan, 79.91129435964582, 89.10313562264948, 68...",15.9,5.300000
3,sub-HC005_ses-MR_BASELINE,"[nan, nan, 165.82130013088184, 182.18457828311...","[nan, 166.75028077088183, 135.1243795211766, 1...","[nan, 130.88847806731278, 113.70680062993898, ...","[nan, 90.33032327850566, 61.05592299498068, 81...","[nan, 103.55341802777106, 82.68805313982449, 9...",12.2,4.066667
4,sub-HC005_ses-MR_FOLLOWUP,"[nan, nan, 119.31195213040063, 115.50987333850...","[nan, 139.18613038571874, 113.36888240550923, ...","[nan, 132.5333467488742, 97.11126116350042, 88...","[nan, 86.47702366515423, 67.25109044285934, 45...","[nan, nan, 63.06734099753868, 59.5852100730314...",12.2,4.066667


### Refine CBF ts data based on transit time, like for dM/M0 data
Again, take the time series list from the above calculation and exclude time points (i.e. PLDs) that happen before ATT (i.e. when all the blood has arrived).

In [10]:
tt_df = pd.read_csv("regionals_tts.csv")
refined_cbf_df = cbf_df.copy()

for row, row in cbf_df.iterrows(): #iterate over rows
    row_list = [row.session_id]
    for c, value in row.items():
        if type(value) == list:
            
            tt = float(tt_df.loc[tt_df['session_id']==row.session_id, c])/1000
 
            # extract post-TT values
            if tt < 1:
                dms = value[0:7]
            elif 1 < tt < 1.22:
                dms = value[1:7] + [0]
            elif 1.22 < tt < 1.48:
                dms = value[2:7] + [0]*2
            elif 1.48 < tt < 1.78:
                dms = value[3:7] + [0]*3
            elif 1.78 < tt < 2.15:
                dms = value[4:7] + [0]*4
            else:
                print("??")

            row_list.append(dms)
            
    row_list.append(row.Hb)
    row_list.append(row.T1a)
    refined_cbf_df.loc[len(refined_cbf_df.index)] = row_list
    
refined_cbf_df = refined_cbf_df.tail(50) # only get the newly created rows
refined_cbf_df.reset_index(drop=True)
refined_cbf_df.to_csv('refined_cbf.csv', index=False)